In [1]:
import pandas as pd

In [2]:
imdb_df = pd.read_csv('dataset/imdb_labelled.txt', sep='\t', header=None)

In [3]:
imdb_df.head()

,0,1
0,A very very very slowmoving aimless movie abou...,0
1,Not sure who was more lost the flat character...,0
2,Attempting artiness with black white and clev...,0
3,Very little music or anything to speak of,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
imdb_df.columns = ['Review', 'Sentiment']

In [5]:
imdb_df.head()

,Review,Sentiment
0,A very very very slowmoving aimless movie abou...,0
1,Not sure who was more lost the flat character...,0
2,Attempting artiness with black white and clev...,0
3,Very little music or anything to speak of,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
def textProcessing(df):
    return final_list